# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, recall_score, f1_score, precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import make_column_selector as selector
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD
from IPython.display import display, Markdown

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Configuration
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "logon_sessions_organization"
FEATURE_SETS  = [
    ["session_duration", "hour"],
    ["session_duration", "weekday"],
    ["session_duration", "hour", "weekday"],
    ["session_duration", "hour_sin", "hour_cos"],
    ["session_duration", "weekday_sin", "weekday_cos"],
    ["session_duration", "hour_sin", "hour_cos", "weekday_sin", "weekday_cos"]
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def train_classifier(df, classifier, features, params, n_iter=10):
    # Create a dataframe where the results of the iterations will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a clone of the classifier
        try:
            classifier = clone(classifier)
        except:
            pass

        # Update the parameters of the classifier according to the grid search
        classifier.set_params(**params)
        
        # Set the random state attribute of the classifier (if it has one)
        try:
            classifier.set_params(**{"random_state": randint(0, 2**32)})
        except Exception:
            pass

        # Define the transformers that do the rest of the preprocessing (scaling, encoding)
        numeric_transformer = Pipeline(steps=[
            ("scaler", RobustScaler())
        ])
        categorical_transformer = Pipeline(steps=[
            ("ohe", OneHotEncoder())
        ])

        # Filter the dataframe according to the features 
        df_filtered = df[features].copy()

        # Create a pipeline that performs the feature selection and scaling
        pipe = Pipeline([
            ("column_transformer", ColumnTransformer(
                transformers=[
                    ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                    ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                ]
            )),
            ("classifier", classifier)
        ])
       
        # Benchmark the training
        start_training = timer()
        pipe.fit(df_filtered)
        end_training = timer()

        # Benchmark the inference
        start_inference = timer()
        pipe.predict(df_filtered)
        end_inference = timer()

        # Calculate different evaluation metrics
        y_true = df["threat"]
        y_pred = pipe.named_steps["classifier"].labels_
        scores = pipe.named_steps["classifier"].decision_scores_
        recall = recall_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(y_true, scores)
            p_auc_10 = roc_auc_score(y_true, scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(y_true, scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(y_true, scores, max_fpr=0.3)
        except ValueError:
            auc = -1

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": end_training - start_training,
                "inference_time": end_inference - start_inference,
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": y_true,
                "y_pred": y_pred,
                "scores": scores
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, features, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.features = features
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None
        

    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=1
                    )
            except ValueError as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, features in enumerate(FEATURE_SETS, start=1):

        display(Markdown(f"# {i}/{len(FEATURE_SETS)} - Features: {', '.join(features)}"))

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=df,
            classifier=classifier,
            features=features,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "logon_sessions"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 24 different hyperparameter combinations
 Training Time: 0.3872    Inference Time: 0.1597    pAUC: 0.6289 ± 0.0777   Recall: 0.0465 ± 0.0929   TN: 469086.0   FP: 1406.0   FN: 76.0   TP: 23.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 2.4101    Inference Time: 1.1180    pAUC: 0.7472 ± 0.0228   Recall: 0.2323 ± 0.0800   TN: 466000.0   FP: 4492.0   FN: 72.0   TP: 27.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 11.5397   Inference Time: 5.7307    pAUC: 0.8039 ± 0.0101   Recall: 0.2707 ± 0.0375   TN: 465820.0   FP: 4672.0   FN: 76.0   TP: 23.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 23.0690   Inference Time: 11.3257   pAUC: 0.8025 ± 0.0134   Recall: 0.2323 ± 0.0202   TN: 465858.0   FP: 4634.0   FN: 77.0   TP: 22.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4352    Inference Time: 0.1910    pAUC: 0.6075 ± 0.0520   Recall: 0.0222 ± 0.0289   TN: 469850.0   FP: 642.0   FN: 92.0   TP: 7.0

# 2/6 - Features: session_duration, weekday

Testing 24 different hyperparameter combinations
 Training Time: 0.3759    Inference Time: 0.1666    pAUC: 0.4970 ± 0.0168   Recall: 0.0061 ± 0.0121   TN: 469146.0   FP: 1346.0   FN: 96.0   TP: 3.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 2.5852    Inference Time: 1.1915    pAUC: 0.5727 ± 0.0264   Recall: 0.0263 ± 0.0244   TN: 466089.0   FP: 4403.0   FN: 93.0   TP: 6.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 11.5263   Inference Time: 5.6355    pAUC: 0.5571 ± 0.0196   Recall: 0.0303 ± 0.0212   TN: 465864.0   FP: 4628.0   FN: 94.0   TP: 5.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 22.5480   Inference Time: 10.7787   pAUC: 0.5522 ± 0.0246   Recall: 0.0222 ± 0.0148   TN: 465850.0   FP: 4642.0   FN: 95.0   TP: 4.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.3790    Inference Time: 0.1683    pAUC: 0.5069 ± 0.0247   Recall: 0.0061 ± 0.0121   TN: 469963.0   FP: 529.0   FN: 99.0   TP: 0.0   P

# 3/6 - Features: session_duration, hour, weekday

...

Testing 24 different hyperparameter combinations
 Training Time: 0.3751    Inference Time: 0.1619    pAUC: 0.6610 ± 0.1328   Recall: 0.0000 ± 0.0000   TN: 470492.0   FP: 0.0     FN: 99.0   TP: 0.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 2.5373    Inference Time: 1.1703    pAUC: 0.8479 ± 0.0159   Recall: 0.4202 ± 0.1160   TN: 467328.0   FP: 3164.0   FN: 60.0   TP: 39.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 11.4541   Inference Time: 5.4441    pAUC: 0.8786 ± 0.0045   Recall: 0.4000 ± 0.0304   TN: 465836.0   FP: 4656.0   FN: 64.0   TP: 35.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 22.9432   Inference Time: 11.0985   pAUC: 0.8824 ± 0.0062   Recall: 0.4364 ± 0.0449   TN: 465827.0   FP: 4665.0   FN: 60.0   TP: 39.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4513    Inference Time: 0.1996    pAUC: 0.6338 ± 0.0766   Recall: 0.0040 ± 0.0081   TN: 470492.0   FP: 0.0     FN: 99.0   TP: 0.0  

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 0.4295    Inference Time: 0.1945    pAUC: 0.5085 ± 0.0397   Recall: 0.0000 ± 0.0000   TN: 470492.0   FP: 0.0     FN: 99.0   TP: 0.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 2.8629    Inference Time: 1.3098    pAUC: 0.5836 ± 0.0209   Recall: 0.0101 ± 0.0111   TN: 465824.0   FP: 4668.0   FN: 99.0   TP: 0.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 13.1321   Inference Time: 6.0549    pAUC: 0.5993 ± 0.0163   Recall: 0.0404 ± 0.0202   TN: 465815.0   FP: 4677.0   FN: 94.0   TP: 5.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 25.6396   Inference Time: 11.9485   pAUC: 0.6062 ± 0.0236   Recall: 0.0505 ± 0.0111   TN: 465823.0   FP: 4669.0   FN: 96.0   TP: 3.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4614    Inference Time: 0.2014    pAUC: 0.5083 ± 0.0242   Recall: 0.0020 ± 0.0040   TN: 469030.0   FP: 1462.0   FN: 98.0   TP: 1.0   P

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 0.4951    Inference Time: 0.2102    pAUC: 0.5867 ± 0.0753   Recall: 0.0000 ± 0.0000   TN: 470492.0   FP: 0.0     FN: 99.0   TP: 0.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 3.0062    Inference Time: 1.3761    pAUC: 0.7632 ± 0.0283   Recall: 0.1616 ± 0.1004   TN: 466535.0   FP: 3957.0   FN: 65.0   TP: 34.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 13.7850   Inference Time: 6.4135    pAUC: 0.7790 ± 0.0210   Recall: 0.1475 ± 0.0870   TN: 465803.0   FP: 4689.0   FN: 87.0   TP: 12.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 27.3642   Inference Time: 12.7119   pAUC: 0.7891 ± 0.0205   Recall: 0.1616 ± 0.1044   TN: 465804.0   FP: 4688.0   FN: 89.0   TP: 10.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.5195    Inference Time: 0.2233    pAUC: 0.6008 ± 0.0722   Recall: 0.0303 ± 0.0373   TN: 466227.0   FP: 4265.0   FN: 91.0   TP: 8.0 

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 28 different hyperparameter combinations
 Training Time: 1.9145    Inference Time: 1.3339    pAUC: 0.7113 ± 0.0000   Recall: 0.1111 ± 0.0000   TN: 468033   FP: 2459    FN: 88    TP: 11    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 3.7630    Inference Time: 1.5790    pAUC: 0.7796 ± 0.0000   Recall: 0.1414 ± 0.0000   TN: 467947   FP: 2545    FN: 85    TP: 14    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.4528    Inference Time: 2.2502    pAUC: 0.7322 ± 0.0000   Recall: 0.1111 ± 0.0000   TN: 468027   FP: 2465    FN: 88    TP: 11    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.5991    Inference Time: 2.9970    pAUC: 0.7322 ± 0.0000   Recall: 0.1111 ± 0.0000   TN: 468027   FP: 2465    FN: 88    TP: 11    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.2465    Inference Time: 0.7771    pAUC: 0.7422 ± 0.0000   Recall: 0.1515 ± 0.0000   TN: 467836   FP: 2656    FN: 84    TP: 15    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 2/6 - Features: session_duration, weekday

Testing 28 different hyperparameter combinations
 Training Time: 1.2639    Inference Time: 0.7428    pAUC: 0.4820 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 470491   FP: 1       FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.3442    Inference Time: 1.4629    pAUC: 0.5705 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 470491   FP: 1       FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.5444    Inference Time: 2.1924    pAUC: 0.5705 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 470491   FP: 1       FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 5.5558    Inference Time: 2.9022    pAUC: 0.5705 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 470491   FP: 1       FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.2505    Inference Time: 0.7456    pAUC: 0.5322 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 470492   FP: 0       FN: 99    TP: 0     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 3/6 - Features: session_duration, hour, weekday

...

Testing 28 different hyperparameter combinations
 Training Time: 1.2251    Inference Time: 0.7618    pAUC: 0.7715 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 467652   FP: 2840    FN: 95    TP: 4     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.3860    Inference Time: 1.5091    pAUC: 0.7793 ± 0.0000   Recall: 0.1414 ± 0.0000   TN: 467637   FP: 2855    FN: 85    TP: 14    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.5394    Inference Time: 2.2042    pAUC: 0.7879 ± 0.0000   Recall: 0.0808 ± 0.0000   TN: 467744   FP: 2748    FN: 91    TP: 8     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.6810    Inference Time: 3.0072    pAUC: 0.7767 ± 0.0000   Recall: 0.0808 ± 0.0000   TN: 467696   FP: 2796    FN: 91    TP: 8     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.2718    Inference Time: 0.7920    pAUC: 0.7994 ± 0.0000   Recall: 0.1616 ± 0.0000   TN: 467888   FP: 2604    FN: 83    TP: 16    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 1.2392    Inference Time: 0.7608    pAUC: 0.5836 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 470492   FP: 0       FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.4152    Inference Time: 1.4462    pAUC: 0.5836 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 470492   FP: 0       FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.5645    Inference Time: 2.1596    pAUC: 0.5836 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 470492   FP: 0       FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.7175    Inference Time: 2.9162    pAUC: 0.4879 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 470492   FP: 0       FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.2677    Inference Time: 0.7713    pAUC: 0.5220 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 468738   FP: 1754    FN: 99    TP: 0     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 1.5212    Inference Time: 0.8953    pAUC: 0.5949 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465788   FP: 4704    FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.5549    Inference Time: 1.6040    pAUC: 0.6111 ± 0.0000   Recall: 0.2424 ± 0.0000   TN: 466523   FP: 3969    FN: 75    TP: 24    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.6012    Inference Time: 2.3127    pAUC: 0.6732 ± 0.0000   Recall: 0.1313 ± 0.0000   TN: 465870   FP: 4622    FN: 86    TP: 13    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.7343    Inference Time: 3.0839    pAUC: 0.6418 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 465819   FP: 4673    FN: 90    TP: 9     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.3246    Inference Time: 0.8182    pAUC: 0.6909 ± 0.0000   Recall: 0.1111 ± 0.0000   TN: 465967   FP: 4525    FN: 88    TP: 11    Params: {'n_bins': 8, 'n_random_cuts': 25}


# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 1 different hyperparameter combinations
 Training Time: 0.5589    Inference Time: 1.1880    pAUC: 0.6739 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465820   FP: 4672    FN: 98    TP: 1     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6739 ± 0.0000
Recall: 0.0101 ± 0.0000


# 2/6 - Features: session_duration, weekday

Testing 1 different hyperparameter combinations
 Training Time: 0.5342    Inference Time: 1.0627    pAUC: 0.5650 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 465792   FP: 4700    FN: 95    TP: 4     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5650 ± 0.0000
Recall: 0.0404 ± 0.0000


# 3/6 - Features: session_duration, hour, weekday

...

Testing 1 different hyperparameter combinations
 Training Time: 0.7544    Inference Time: 1.4343    pAUC: 0.7355 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 465805   FP: 4687    FN: 90    TP: 9     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7355 ± 0.0000
Recall: 0.0909 ± 0.0000


# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 0.7054    Inference Time: 1.3428    pAUC: 0.5460 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 466249   FP: 4243    FN: 99    TP: 0     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5460 ± 0.0000
Recall: 0.0000 ± 0.0000


# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.0432    Inference Time: 2.2307    pAUC: 0.7244 ± 0.0000   Recall: 0.1010 ± 0.0000   TN: 465823   FP: 4669    FN: 89    TP: 10    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7244 ± 0.0000
Recall: 0.1010 ± 0.0000


# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION,
    n_jobs=1
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 1 different hyperparameter combinations
 Training Time: 0.5540    Inference Time: 1.1132    pAUC: 0.6469 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465820   FP: 4672    FN: 99    TP: 0     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6469 ± 0.0000
Recall: 0.0000 ± 0.0000


# 2/6 - Features: session_duration, weekday

Testing 1 different hyperparameter combinations
 Training Time: 0.5378    Inference Time: 1.0497    pAUC: 0.5060 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465842   FP: 4650    FN: 98    TP: 1     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5060 ± 0.0000
Recall: 0.0101 ± 0.0000


# 3/6 - Features: session_duration, hour, weekday

...

Testing 1 different hyperparameter combinations
 Training Time: 0.7346    Inference Time: 1.4373    pAUC: 0.7064 ± 0.0000   Recall: 0.0808 ± 0.0000   TN: 465864   FP: 4628    FN: 91    TP: 8     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7064 ± 0.0000
Recall: 0.0808 ± 0.0000


# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 0.6870    Inference Time: 1.3098    pAUC: 0.4829 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465815   FP: 4677    FN: 99    TP: 0     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.4829 ± 0.0000
Recall: 0.0000 ± 0.0000


# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.0799    Inference Time: 1.9680    pAUC: 0.6974 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 465809   FP: 4683    FN: 95    TP: 4     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6974 ± 0.0000
Recall: 0.0404 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [2, 4],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 80 different hyperparameter combinations
 Training Time: 2.1859    Inference Time: 0.1057    pAUC: 0.7373 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 465833   FP: 4659    FN: 90    TP: 9     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 1.6156    Inference Time: 0.0722    pAUC: 0.7430 ± 0.0000   Recall: 0.1010 ± 0.0000   TN: 465869   FP: 4623    FN: 89    TP: 10    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 2.3748    Inference Time: 0.0812    pAUC: 0.4737 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465799   FP: 4693    FN: 99    TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 2.4007    Inference Time: 0.0831    pAUC: 0.7496 ± 0.0000   Recall: 0.1010 ± 0.0000   TN: 465869   FP: 4623    FN: 89    TP: 10    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 1.6641    Inference Time: 0.0767    pAUC: 0.7373 ± 0.0000

# 2/6 - Features: session_duration, weekday

Testing 80 different hyperparameter combinations
 Training Time: 1.7905    Inference Time: 0.0738    pAUC: 0.4775 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465789   FP: 4703    FN: 99    TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 1.7966    Inference Time: 0.0709    pAUC: 0.5826 ± 0.0004   Recall: 0.0404 ± 0.0000   TN: 466442   FP: 4050    FN: 95    TP: 4     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 2.9962    Inference Time: 0.0810    pAUC: 0.4737 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465798   FP: 4694    FN: 99    TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 3.4050    Inference Time: 0.0775    pAUC: 0.6488 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 465801   FP: 4691    FN: 95    TP: 4     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 1.7500    Inference Time: 0.0828    pAUC: 0.4775 ± 0.0000

# 3/6 - Features: session_duration, hour, weekday

...

Testing 80 different hyperparameter combinations
 Training Time: 1.7907    Inference Time: 0.1021    pAUC: 0.6753 ± 0.0000   Recall: 0.1717 ± 0.0000   TN: 465803   FP: 4689    FN: 82    TP: 17    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 1.7977    Inference Time: 0.0940    pAUC: 0.7415 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465861   FP: 4631    FN: 97    TP: 2     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 2.5826    Inference Time: 0.0985    pAUC: 0.4831 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465790   FP: 4702    FN: 99    TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 2.5379    Inference Time: 0.0954    pAUC: 0.7412 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465860   FP: 4632    FN: 97    TP: 2     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 1.9412    Inference Time: 0.1145    pAUC: 0.6753 ± 0.0000

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 80 different hyperparameter combinations
 Training Time: 1.7634    Inference Time: 0.0922    pAUC: 0.5480 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 466347   FP: 4145    FN: 99    TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 1.6981    Inference Time: 0.0897    pAUC: 0.4930 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 466347   FP: 4145    FN: 99    TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 3.1231    Inference Time: 0.0983    pAUC: 0.4990 ± 0.0207   Recall: 0.0242 ± 0.0198   TN: 466553   FP: 3939    FN: 95    TP: 4     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 3.1464    Inference Time: 0.0941    pAUC: 0.6023 ± 0.0198   Recall: 0.0364 ± 0.0049   TN: 466018   FP: 4474    FN: 96    TP: 3     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 1.7614    Inference Time: 0.0957    pAUC: 0.5480 ± 0.0000

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 80 different hyperparameter combinations
 Training Time: 2.1086    Inference Time: 0.1242    pAUC: 0.6245 ± 0.0000   Recall: 0.1414 ± 0.0000   TN: 465801   FP: 4691    FN: 85    TP: 14    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.0931    Inference Time: 0.1161    pAUC: 0.7371 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465803   FP: 4689    FN: 97    TP: 2     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 3.2392    Inference Time: 0.1298    pAUC: 0.5274 ± 0.0000   Recall: 0.0505 ± 0.0000   TN: 465830   FP: 4662    FN: 94    TP: 5     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 3.2447    Inference Time: 0.1243    pAUC: 0.7708 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465795   FP: 4697    FN: 97    TP: 2     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.1033    Inference Time: 0.1192    pAUC: 0.6245 ± 0.0000

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 168 different hyperparameter combinations
 Training Time: 0.1393    Inference Time: 0.0540    pAUC: 0.7417 ± 0.0000   Recall: 0.1111 ± 0.0000   TN: 465800   FP: 4692    FN: 88    TP: 11    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.1289    Inference Time: 0.0543    pAUC: 0.7417 ± 0.0000   Recall: 0.1111 ± 0.0000   TN: 465800   FP: 4692    FN: 88    TP: 11    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.1340    Inference Time: 0.0567    pAUC: 0.7417 ± 0.0000   Recall: 0.1111 ± 0.0000   TN: 465800   FP: 4692    FN: 88    TP: 11    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.1313    Inference Time: 0.0558    pAUC: 0.7417 ± 0.0000   Recall: 0.1111 ± 0.0000   TN: 465800   FP: 4692    FN: 88    TP: 11    Params: {'n_components': 1, 'stan

# 2/6 - Features: session_duration, weekday

Testing 168 different hyperparameter combinations
 Training Time: 0.1275    Inference Time: 0.0533    pAUC: 0.4771 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465871   FP: 4621    FN: 99    TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.1258    Inference Time: 0.0560    pAUC: 0.4771 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465871   FP: 4621    FN: 99    TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.1283    Inference Time: 0.0551    pAUC: 0.4771 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465871   FP: 4621    FN: 99    TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.1283    Inference Time: 0.0542    pAUC: 0.4771 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465871   FP: 4621    FN: 99    TP: 0     Params: {'n_components': 1, 'stan

# 3/6 - Features: session_duration, hour, weekday

...

Testing 168 different hyperparameter combinations
 Training Time: 0.1800    Inference Time: 0.0663    pAUC: 0.7570 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465872   FP: 4620    FN: 98    TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.1864    Inference Time: 0.0661    pAUC: 0.7570 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465872   FP: 4620    FN: 98    TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.1780    Inference Time: 0.0685    pAUC: 0.7570 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465872   FP: 4620    FN: 98    TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.1789    Inference Time: 0.0695    pAUC: 0.7570 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465872   FP: 4620    FN: 98    TP: 1     Params: {'n_components': 1, 'stan

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 168 different hyperparameter combinations
 Training Time: 0.1827    Inference Time: 0.0689    pAUC: 0.6645 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 466020   FP: 4472    FN: 98    TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.1844    Inference Time: 0.0712    pAUC: 0.6645 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 466020   FP: 4472    FN: 98    TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.1802    Inference Time: 0.0698    pAUC: 0.6645 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 466020   FP: 4472    FN: 98    TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.1781    Inference Time: 0.0687    pAUC: 0.6645 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 466020   FP: 4472    FN: 98    TP: 1     Params: {'n_components': 1, 'stan

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 168 different hyperparameter combinations
 Training Time: 0.2792    Inference Time: 0.0821    pAUC: 0.7540 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465796   FP: 4696    FN: 97    TP: 2     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.2847    Inference Time: 0.0834    pAUC: 0.7540 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465796   FP: 4696    FN: 97    TP: 2     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.2790    Inference Time: 0.0825    pAUC: 0.7540 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465796   FP: 4696    FN: 97    TP: 2     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.2782    Inference Time: 0.0840    pAUC: 0.7540 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465796   FP: 4696    FN: 97    TP: 2     Params: {'n_components': 1, 'stan

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 80.0737   Inference Time: 18.0208   pAUC: 0.7767 ± 0.0002   Recall: 0.1010 ± 0.0000   TN: 465862   FP: 4630    FN: 89    TP: 10    Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 79.6855   Inference Time: 17.3349   pAUC: 0.7769 ± 0.0001   Recall: 0.1010 ± 0.0000   TN: 465869   FP: 4623    FN: 89    TP: 10    Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 91.8599   Inference Time: 19.3980   pAUC: 0.7761 ± 0.0011   Recall: 0.1010 ± 0.0000   TN: 465869   FP: 4623    FN: 89    TP: 10    Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 88.4300  

# 2/6 - Features: session_duration, weekday

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 91.1311   Inference Time: 16.2932   pAUC: 0.5338 ± 0.0237   Recall: 0.0061 ± 0.0121   TN: 465937   FP: 4555    FN: 96    TP: 3     Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 81.4886   Inference Time: 18.0535   pAUC: 0.5387 ± 0.0204   Recall: 0.0061 ± 0.0121   TN: 466041   FP: 4451    FN: 96    TP: 3     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 101.6701   Inference Time: 16.1588   pAUC: 0.5732 ± 0.0003   Recall: 0.0303 ± 0.0000   TN: 465808   FP: 4684    FN: 96    TP: 3     Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 100.7855

# 3/6 - Features: session_duration, hour, weekday

...

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 75.5124   Inference Time: 19.2974   pAUC: 0.7325 ± 0.0024   Recall: 0.1131 ± 0.0358   TN: 465801   FP: 4691    FN: 93    TP: 6     Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 94.6247   Inference Time: 14.3182   pAUC: 0.7305 ± 0.0012   Recall: 0.1434 ± 0.0364   TN: 465910   FP: 4582    FN: 92    TP: 7     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 102.1712   Inference Time: 18.0680   pAUC: 0.7445 ± 0.0003   Recall: 0.0202 ± 0.0000   TN: 465859   FP: 4633    FN: 97    TP: 2     Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 88.7610 

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 92.4220   Inference Time: 15.3792   pAUC: 0.6255 ± 0.0134   Recall: 0.0384 ± 0.0040   TN: 466018   FP: 4474    FN: 96    TP: 3     Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 87.2317   Inference Time: 16.9846   pAUC: 0.6582 ± 0.0059   Recall: 0.0303 ± 0.0000   TN: 466018   FP: 4474    FN: 96    TP: 3     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 94.2338   Inference Time: 18.4766   pAUC: 0.6075 ± 0.0044   Recall: 0.0404 ± 0.0000   TN: 465828   FP: 4664    FN: 95    TP: 4     Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 93.2375  

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
 Training Time: 125.8102   Inference Time: 18.0842   pAUC: 0.7494 ± 0.0057   Recall: 0.0687 ± 0.0768   TN: 465810   FP: 4682    FN: 96    TP: 3     Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 102.5839   Inference Time: 14.3779   pAUC: 0.7472 ± 0.0060   Recall: 0.1899 ± 0.0196   TN: 465827   FP: 4665    FN: 79    TP: 20    Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 99.5113   Inference Time: 18.4294   pAUC: 0.7532 ± 0.0012   Recall: 0.0545 ± 0.0485   TN: 465802   FP: 4690    FN: 96    TP: 3     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 128.9208   Inference Time: 17.9825   pAUC: 0.7524 ± 0.0004   Recall: 0.0303 ± 0.0000   TN: 465793   FP: 4699    FN: 96    TP: 3     Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 107.1258   Inference Time: 19.3960   pAUC: 0.7523 ± 0.0003   R

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 77.9515   Inference Time: 12.6089   pAUC: 0.7324 ± 0.0452   Recall: 0.1253 ± 0.0675   TN: 465818   FP: 4674    FN: 89    TP: 10    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_1088" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_1088/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_1088/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 

# 2/6 - Features: session_duration, weekday

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 80.9005   Inference Time: 16.7264   pAUC: 0.5284 ± 0.0293   Recall: 0.0000 ± 0.0000   TN: 465789   FP: 4703    FN: 99    TP: 0     Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_1211" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_1211/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_1211/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 

# 3/6 - Features: session_duration, hour, weekday

...

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 59.4311   Inference Time: 12.7927   pAUC: 0.6985 ± 0.0529   Recall: 0.1131 ± 0.0686   TN: 465796   FP: 4696    FN: 89    TP: 10    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_1457" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_1457/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_1457/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 83.2231   Inference Time: 14.4388   pAUC: 0.6033 ± 0.0628   Recall: 0.0343 ± 0.0208   TN: 465818   FP: 4674    FN: 93    TP: 6     Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_1580" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_1580/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_1580/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 84.3625   Inference Time: 12.4530   pAUC: 0.6896 ± 0.0391   Recall: 0.1131 ± 0.0749   TN: 465804   FP: 4688    FN: 96    TP: 3     Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_1703" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_1703/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_1703/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 